In [ ]:
import json

from langchain.chains.question_answering import load_qa_chain

from langchain_community.llms import Ollama
# from langchain_community.embeddings.laser import LaserEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatMaritalk

from langchain_core.prompts.chat import ChatPromptTemplate

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings

# from langchain.experimental.text_splitter import SemanticChunker

import numpy as np

import pandas as pd

- langchain                                0.3.21
- langchain-community                      0.3.20
- langchain-core                           0.3.49
- langchain-huggingface                    0.0.3
- langchain-text-splitters                 0.3.7
- maritalk                                 0.2.6
- pandas                                   2.2.3

    ```
    pip install langchain==0.2.0 langchain-core langchain-huggingface langchain-community
    ```
    

In [4]:
# obtains the API key used to access Maritalk's API
with open('API_KEY.json', 'r') as file:
    maritalk = json.load(file)

model = ChatMaritalk(
    model='sabia-3',
    api_key=maritalk['key'],
    temperature=0.7,
    max_tokens=500,
)

In [3]:
# ollama_base_url= 'http://tempestade.facom.ufms.br:11435'
ollama_base_url = 'http://localhost:11434' 
# model = Ollama(base_url=ollama_base_url, model="phi3:medium", temperature=0)

# ollama_base_url = 'http://localhost:11434' 
model = Ollama(base_url=ollama_base_url, model="llama3.1", temperature=0)

# embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
embeddings_model = HuggingFaceEmbeddings(model_name='stjiris/bert-large-portuguese-cased-legal-tsdae-gpl-nli-sts-MetaKD-v0')

/tmp/ipykernel_412132/2330160043.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(base_url=ollama_base_url, model="llama3.1", temperature=0)
/home/lucass/.miniconda3/envs/semantic-rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Invalid model-index. Not loading eval results into CardData.
Invalid model-index. Not loading eval results into CardData.


In [29]:
doc_folder = 'edital1'

In [27]:
documents_dir_path = f'./database/{doc_folder}'

loader = DirectoryLoader(documents_dir_path, glob='./*.pdf', loader_cls=PyPDFLoader)

loaded_pdfs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=250,
        separators=['\n', '.']
)

# text_splitter = 

pages = text_splitter.split_documents(loaded_pdfs)

In [28]:
vectorstore = FAISS.from_documents(
    pages,
    embeddings_model
)

vectorstore.save_local(f'database/{doc_folder}/document_index')

In [30]:
# loads vectorstore from disk
vectorstore = FAISS.load_local(f'database/{doc_folder}/document_index', embeddings_model, allow_dangerous_deserialization=True)

In [31]:
retriever = vectorstore.as_retriever(search_kwargs={'k':20})

In [32]:
def assistant(content: str):
    return ("assistant", content)

def user(content: str):
    return ("user", content)

In [33]:
prompt_structure = '''
        Baseado nos seguintes documentos:
        {context}
        Responda a pergunta abaixo:
        {query}
        '''

In [10]:
# role based prompt
qa_prompt = ChatPromptTemplate.from_messages([
        user('''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita INTEGRALMENTE o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Você SEMPRE cita o número do item que contém a resposta.'''),
        ('human', prompt_structure)
])

In [34]:
# few-shot prompting
qa_prompt = ChatPromptTemplate.from_messages([
        user('''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita INTEGRALMENTE o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Você SEMPRE cita o número do item que contém a resposta. Aqui estão alguns exemplos:'''),
        user('''Como será a lista de espera?'''),
        assistant('''De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.'''),
        user('''Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?'''),
        assistant('''De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.'''),
        user(prompt_structure)
])

In [35]:
chain = load_qa_chain(model, chain_type='stuff', verbose=True, prompt=qa_prompt)

In [16]:
query = input()

In [36]:
# retrieval_result = retriver.invoke(query)
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

retrieval_result = compression_retriever.invoke(query)


NameError: name 'query' is not defined

In [37]:
def reranker_retrieval(query):
    return compression_retriever.invoke(query)

def vectorstore_retrieval(query):
    retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10, score_threshold=0.5)
    return [page[0] for page in retrieval_result]

In [ ]:
retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10, score_threshold=0.5)

# retrieved context, no enrichment SOMENTE USADO NA BUSCA DIRETO PELA VECTOR STORE
retrieval_result= [page[0] for page in retrieval_result]

In [ ]:
# enriching the retrieval result PARENT DOCUMENT RETRIEVAL

# get the page index of the page with the most similar chunk
# page_idx = retrieval_result[0][0].metadata['page']

# retrieval_result = np.concatenate(
#     (
#         [x for x in pages if x.metadata['page'] == page_idx],
#         [page[0] for page in retrieval_result]),
#     axis=0
# )

In [21]:
qa_prompt

ChatPromptTemplate(input_variables=['context', 'query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento\n         de todo o documento apresentado como contexto e \n         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.\n         Você sempre cita INTEGRALMENTE o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". \n         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Você SEMPRE cita o número do item que contém a resposta. Aqui estão alguns exemplos:')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Como será a lista de espera?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='De acordo 

In [17]:
output = chain.invoke(
        {'input_documents':retrieval_result, 'query':query}
)

print(output['output_text'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita INTEGRALMENTE o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Você SEMPRE cita o número do item que contém a resposta. Aqui estão alguns exemplos:
Human: Como será a lista de espera?
AI: De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.
Human: Q

### Geração das respostas

In [16]:
map_editais = {70 : 'edital1', 9 : 'edital2'}

In [38]:
queries = pd.read_csv('tests/test_questions.csv', dtype={'numero':int})

enrichment = False

answers = []

for num_doc in [70, 9]:

    questions = queries[queries['numero'] == num_doc]['pergunta'].tolist()

    for question in questions:
        
        # query = f'De acordo com o edital {num_doc}. ' + question

        vectorstore = FAISS.load_local(f'database/{map_editais[num_doc]}/document_index', embeddings_model, allow_dangerous_deserialization=True)

        retrieval_result = reranker_retrieval(question)

        if enrichment:
            # enriching the retrieval result 

            # get the page index of the page with the most similar chunk
            page_idx = retrieval_result[0][0].metadata['page']

            retrieval_result = np.concatenate(
                (
                    [x for x in pages if x.metadata['page'] == page_idx],
                    [page[0] for page in retrieval_result]),
                axis=0
            )
        else:
            # retrieved context, no enrichment
            # retrieval_result= [page[0] for page in retrieval_result]
            pass

        output = chain.invoke(
            {'input_documents':retrieval_result, 'query':question}
        )

        answers.append(output['output_text'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita INTEGRALMENTE o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Você SEMPRE cita o número do item que contém a resposta. Aqui estão alguns exemplos:
Human: Como será a lista de espera?
AI: De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.
Human: Q

In [39]:
result = pd.DataFrame(
    {
        'pergunta':queries['pergunta'].tolist(),
        'resposta':answers,
        'edital':queries['arquivo'].tolist()
    }
)

In [40]:
result.to_csv(f'tests/results/{model.model}-few-shot-reranker-bertimbau.csv', index=False)